In [1]:
import os

In [2]:
%pwd

'd:\\ML Projects\\End-to-End-Red-Wine-Quality-Prediction-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\ML Projects\\End-to-End-Red-Wine-Quality-Prediction-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir : Path
    data_path : Path

In [6]:
from Red_Wine_Quality_Prediction.constants import *
from Red_Wine_Quality_Prediction.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
        )

        return data_transformation_config

In [14]:
import os
from Red_Wine_Quality_Prediction import logger
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn import preprocessing
from imblearn.over_sampling import SMOTE
from numpy import log

In [15]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config


    def preprocess_data(self):

        oversample = SMOTE()
        data = pd.read_csv(self.config.data_path)
        data = data.drop('Id', axis=1)

        features, labels = oversample.fit_resample(data.drop('quality', axis=1), data.quality)

        scaler = preprocessing.MinMaxScaler()
        scaled_data = pd.DataFrame(scaler.fit_transform(features), columns=features.columns)

        normalized_arr = preprocessing.normalize(scaled_data)
        normalized_data = pd.DataFrame(normalized_arr, columns=features.columns)

        unskew_data = normalized_data.copy(deep=True)
        unskew_data['residual sugar'] = unskew_data['residual sugar'].replace(0.0, 0.01).apply(log)
        unskew_data['chlorides'] = unskew_data['chlorides'].replace(0.0, 0.01).apply(log)
        unskew_data['total sulfur dioxide'] = unskew_data['total sulfur dioxide'].replace(0.0, 0.01).apply(log)
        unskew_data['free sulfur dioxide'] = unskew_data['free sulfur dioxide'].replace(0.0, 0.01).apply(log)

        unskew_data['quality'] = labels

        unskew_data.to_csv(os.path.join(self.config.root_dir, "preprocessed_data.csv"), index=False)        


    def train_test_spliting(self):
        data = pd.read_csv(self.config.preprocessed_data_path)

        # Split the data into training and test sets. (0.75, 0.25) split.
        train, test = train_test_split(data)

        train.to_csv(os.path.join(self.config.root_dir, "train.csv"),index = False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"),index = False)

        logger.info("Splited data into training and test sets")
        logger.info(train.shape)
        logger.info(test.shape)

        print(train.shape)
        print(test.shape)

In [16]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.preprocess_data()
    data_transformation.train_test_spliting()
except Exception as e:
    raise e

[2024-07-02 12:08:34,054: INFO: common : yaml file: config\config.yaml loaded successfully]
[2024-07-02 12:08:34,057: INFO: common : yaml file: params.yaml loaded successfully]
[2024-07-02 12:08:34,060: INFO: common : yaml file: schema.yaml loaded successfully]
[2024-07-02 12:08:34,061: INFO: common : created directory at: artifacts]
[2024-07-02 12:08:34,062: INFO: common : created directory at: artifacts/data_transformation]


FileNotFoundError: [Errno 2] No such file or directory: 'artifacts/data_ingestion/WineQT.csv'